<a href="https://colab.research.google.com/github/angiepaloes/Sistemas-avanzados-de-producci-n-con-phyton/blob/main/Copia_de_Clase_3_Taller_1_Datos_Con_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Clase 3 (2h): Muestreo, bootstrap, TCL y regresión lineal

**Objetivos**
- Muestra vs. población y variabilidad muestral.
- Bootstrap para intervalo de confianza (media).
- Teorema Central del Límite (TCL) con simulaciones.
- Correlación de Pearson y regresión lineal.
- Paradoja de Simpson (demostración).


In [ ]:

import sys, platform
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
%matplotlib inline
print("Python:", sys.version.split()[0], "|", platform.platform())


In [ ]:

# Dataset base (si no existe 'da')
if 'da' not in globals():
    np.random.seed(123)
    n = 1000
    da = pd.DataFrame({
        "SEQN": np.arange(1, n+1),
        "RIAGENDR": np.random.choice([1, 2], size=n),
        "RIDAGEYR": np.random.randint(18, 80, size=n),
        "BMXWT": np.round(np.random.normal(72, 14, size=n), 1),
        "BMXHT": np.round(np.random.normal(169, 9, size=n), 1),
    })
    da["BMXBMI"] = np.round(da["BMXWT"] / (da["BMXHT"]/100)**2, 1)
    da["sexo"] = da["RIAGENDR"].map({1:"Masculino", 2:"Femenino"})
da.head()

## Distribución de medias muestrales

In [ ]:

np.random.seed(7)
base = da["BMXBMI"].dropna().values if "BMXBMI" in da.columns else da.select_dtypes(include=[np.number]).iloc[:,0].dropna().values

def medias_muestrales(base, n_muestra=60, repeticiones=800):
    medios = []
    for _ in range(repeticiones):
        muestra = np.random.choice(base, size=n_muestra, replace=True)
        medios.append(muestra.mean())
    return np.array(medios)

mm = medias_muestrales(base, n_muestra=60, repeticiones=800)
print("Media base:", np.mean(base).round(3), "| Media de medias:", mm.mean().round(3))

plt.figure(); plt.hist(mm, bins=30)
plt.title("Distribución de medias muestrales (n=60)")
plt.xlabel("Media muestral"); plt.ylabel("Frecuencia"); plt.show()

## Bootstrap: IC para la media

In [ ]:

def bootstrap_ci_media(x, B=2000, conf=0.95):
    x = np.asarray(x); n = len(x)
    boots = np.empty(B)
    for b in range(B):
        idx = np.random.randint(0, n, n)
        boots[b] = x[idx].mean()
    alpha = (1-conf)/2
    lo, hi = np.quantile(boots, [alpha, 1-alpha])
    return lo, hi, boots

lo, hi, boots = bootstrap_ci_media(base, B=2000, conf=0.95)
print(f"IC 95% bootstrap ≈ [{lo:.2f}, {hi:.2f}]")

plt.figure(); plt.hist(boots, bins=30)
plt.title("Bootstrap de la media"); plt.xlabel("Media bootstrap"); plt.ylabel("Frecuencia"); plt.show()


## Teorema Central del Límite (TCL)

In [ ]:

np.random.seed(33); lam = 1.0; Nrep = 2000
for n in [5, 30, 100]:
    means = [np.mean(np.random.exponential(scale=1/lam, size=n)) for _ in range(Nrep)]
    plt.figure(); plt.hist(means, bins=30)
    plt.title(f"TCL: medias con exponencial (n={n})")
    plt.xlabel("Media muestral"); plt.ylabel("Frecuencia"); plt.show()


## Correlación y regresión lineal (Edad → IMC)

In [ ]:

if {"RIDAGEYR","BMXBMI"}.issubset(da.columns):
    x = da["RIDAGEYR"].values; y = da["BMXBMI"].values
    r, p = stats.pearsonr(x, y)
    print(f"Correlación Pearson (Edad, IMC): r={r:.3f}, p={p:.3g}")
    X = sm.add_constant(x)
    model = sm.OLS(y, X, missing='drop').fit()
    print(model.summary())
    xgrid = np.linspace(x.min(), x.max(), 100); yhat = model.predict(sm.add_constant(xgrid))
    plt.figure(); plt.scatter(x, y, s=10); plt.plot(xgrid, yhat)
    plt.title("IMC vs Edad con recta OLS"); plt.xlabel("Edad (años)"); plt.ylabel("IMC"); plt.show()
else:
    print("Faltan columnas 'RIDAGEYR' y/o 'BMXBMI'.")


## Paradoja de Simpson

In [ ]:

np.random.seed(2024)
nA, nB = 200, 200
edad_A = np.random.normal(30, 6, size=nA)
imc_A  = 0.15*edad_A + np.random.normal(0, 2.5, size=nA) + 20
edad_B = np.random.normal(55, 6, size=nB)
imc_B  = 0.15*edad_B + np.random.normal(0, 2.5, size=nB) + 10

edad = np.concatenate([edad_A, edad_B])
imc  = np.concatenate([imc_A, imc_B])

XA = sm.add_constant(edad_A); XB = sm.add_constant(edad_B)
mA = sm.OLS(imc_A, XA).fit(); mB = sm.OLS(imc_B, XB).fit()
Xg = sm.add_constant(edad); mg = sm.OLS(imc, Xg).fit()

xg = np.linspace(edad.min(), edad.max(), 100)
plt.figure(); plt.scatter(edad_A, imc_A, s=10); plt.plot(xg, mA.predict(sm.add_constant(xg)))
plt.title("Grupo A: IMC vs Edad"); plt.xlabel("Edad"); plt.ylabel("IMC"); plt.show()

plt.figure(); plt.scatter(edad_B, imc_B, s=10); plt.plot(xg, mB.predict(sm.add_constant(xg)))
plt.title("Grupo B: IMC vs Edad"); plt.xlabel("Edad"); plt.ylabel("IMC"); plt.show()

plt.figure(); plt.scatter(edad, imc, s=10); plt.plot(xg, mg.predict(sm.add_constant(xg)))
plt.title("Combinados: IMC vs Edad (Simpson)"); plt.xlabel("Edad"); plt.ylabel("IMC"); plt.show()



## Ejercicios (para tu repositorio)
1) IC 90% por bootstrap para la **mediana** y comparación con el IC de la media.  
2) TCL con otra distribución (gamma/lognormal) y n={10, 40, 120}.  
3) Correlación entre dos pares de variables (interpretación de magnitud, signo, valor-p).  
4) Regresión lineal con otra respuesta y análisis de residuos.



### Respuestas del estudiante (Markdown)
<div class="response-box">
Escribe aquí tu análisis y conclusiones para los puntos 1–4.
</div>


In [ ]:
# Respuestas del estudiante (Código)
# TODO: Añade aquí tu código para los ejercicios de la Clase 3